In [ ]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

In [ ]:
class Clustering:
    def __init__(self, space_size=400, r_sen=50, max_cluster_size=20, min_cluster_size=5):
        self.space_size = space_size
        self.r_sen = r_sen
        self.max_cluster_size = max_cluster_size
        self.min_cluster_size = min_cluster_size

    def estimate_optimal_k(self, nodes, base_station=(200,200,400)):
        """
        Ước tính số cụm tối ưu dựa trên công thức WSN
        K = sqrt(N*L / (pi*d_tobs))
        """
        N = len(nodes)
        base_pos = np.array(base_station)

        # Khoang cach trung binh toi base station
        distances = np.linalg.norm(nodes - base_pos, axis=1)
        d_tobs = np.mean(distances)

        space_size = self.space_size

        k_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
        k_optimal = max(2, int(np.round(k_optimal)))

        # Điều chỉnh dựa trên max_cluster_size
        k_min = int(np.ceil(N / self.max_cluster_size))
        k_optimal = max(k_optimal, k_min)
        
        return k_optimal
    
    def check_cluster_validity(self, cluster_nodes):
        """
        Kiem tra tinh hop le cua cum
        """
        size = len(cluster_nodes)

        # Kiểm tra kích thước
        if size < self.min_cluster_size or size > self.max_cluster_size:
            return False, 0, size
        
        # Kiểm tra khoảng cách
        if size > 1:
            distances = pdist(cluster_nodes)
            max_dist = np.max(distances)
            
            if max_dist > self.r_sen:
                return False, max_dist, size
            
            return True, max_dist, size
        
        return True, 0, size
    
    def split_invalid_cluster(self, cluster_nodes, cluster_ids):
        """
        Chia nhỏ cụm không hợp lệ thành các cụm con
        """
        # Nếu cụm chỉ có 1 node, không thể chia
        if len(cluster_nodes) < 2:
            return [(cluster_nodes, cluster_ids)]
        
        # Sử dụng K-Means để chia 2
        kmeans = KMeans(n_clusters=2, n_init=20, random_state=42)
        labels = kmeans.fit_predict(cluster_nodes)
        
        sub_clusters = []
        for i in range(2):
            sub_nodes = cluster_nodes[labels == i]
            sub_ids = [cluster_ids[j] for j in range(len(cluster_ids)) if labels[j] == i]
            
            if len(sub_nodes) > 0:
                sub_clusters.append((sub_nodes, sub_ids))
        
        return sub_clusters
    
    def merge_small_clusters(self, clusters_data):
        """
        Gộp các cụm nhỏ với cụm láng giềng gần nhất
        """
        if len(clusters_data) <= 1:
            return clusters_data
        
        merged = []
        to_merge = []
        
        # Tìm các cụm nhỏ
        for nodes, ids in clusters_data:
            if len(nodes) < self.min_cluster_size:
                to_merge.append((nodes, ids))
            else:
                merged.append((nodes, ids))
        
        # Gộp từng cụm nhỏ vào cụm gần nhất
        for small_nodes, small_ids in to_merge:
            if len(merged) == 0:
                merged.append((small_nodes, small_ids))
                continue
            
            # Tìm cụm gần nhất
            small_center = np.mean(small_nodes, axis=0)
            min_dist = float('inf')
            best_idx = 0
            
            for i, (nodes, ids) in enumerate(merged):
                center = np.mean(nodes, axis=0)
                dist = np.linalg.norm(small_center - center)
                
                # Kiểm tra xem gộp có vượt quá max_size không
                if dist < min_dist and len(nodes) + len(small_nodes) <= self.max_cluster_size:
                    min_dist = dist
                    best_idx = i
            
            # Gộp
            merged[best_idx] = (
                np.vstack([merged[best_idx][0], small_nodes]),
                merged[best_idx][1] + small_ids
            )
        
        return merged
    
    def cluster_with_constraints(self, nodes, node_ids, k=None, max_iterations=10):
        """
        Phân cụm với ràng buộc - Thuật toán chính
        
        Args:
            nodes: Tọa độ 3D của nodes
            node_ids: ID của nodes
            k: Số cụm (nếu None sẽ tự động ước tính)
            max_iterations: Số lần lặp tối đa để điều chỉnh
            
        Returns:
            List of (cluster_nodes, cluster_ids)
        """
        if k is None:
            k = self.estimate_optimal_k(nodes)
        
        print(f"Bắt đầu phân cụm với k={k}")
        
        # Bước 1: K-Means ban đầu
        kmeans = KMeans(n_clusters=k, n_init=30, random_state=42)
        labels = kmeans.fit_predict(nodes)
        
        # Bước 2: Tạo các cụm và kiểm tra
        iteration = 0
        while iteration < max_iterations:
            print(f"  Vòng lặp {iteration + 1}/{max_iterations}")
            
            valid_clusters = []
            invalid_clusters = []
            
            # Phân loại cụm hợp lệ và không hợp lệ
            for i in range(k):
                cluster_nodes = nodes[labels == i]
                cluster_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
                
                if len(cluster_nodes) == 0:
                    continue
                
                is_valid, max_dist, size = self.check_cluster_validity(cluster_nodes)
                
                if is_valid:
                    valid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✓ hợp lệ (size={size}, max_dist={max_dist:.1f}m)")
                else:
                    invalid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✗ không hợp lệ (size={size}, max_dist={max_dist:.1f}m)")
            
            # Nếu tất cả hợp lệ, kết thúc
            if len(invalid_clusters) == 0:
                print(f"  → Tất cả cụm hợp lệ!")
                break
            
            # Bước 3: Xử lý các cụm không hợp lệ
            for cluster_nodes, cluster_ids in invalid_clusters:
                size = len(cluster_nodes)
                
                if size > self.max_cluster_size:
                    # Cụm quá lớn → Chia nhỏ
                    print(f"    → Chia cụm (size={size})")
                    sub_clusters = self.split_invalid_cluster(cluster_nodes, cluster_ids)
                    valid_clusters.extend(sub_clusters)
                else:
                    # Cụm có khoảng cách quá lớn → Chia nhỏ
                    print(f"    → Chia cụm (khoảng cách lớn)")
                    sub_clusters = self.split_invalid_cluster(cluster_nodes, cluster_ids)
                    valid_clusters.extend(sub_clusters)
            
            # Cập nhật labels và k cho vòng lặp tiếp theo
            k = len(valid_clusters)
            
            # Tạo lại labels từ valid_clusters
            labels = np.zeros(len(nodes), dtype=int)
            for cluster_idx, (_, cluster_ids) in enumerate(valid_clusters):
                for node_id in cluster_ids:
                    node_idx = node_ids.index(node_id)
                    labels[node_idx] = cluster_idx
            
            iteration += 1
        
        # Bước 4: Gộp các cụm quá nhỏ
        valid_clusters = self.merge_small_clusters(valid_clusters)
        
        print(f"Hoàn thành: {len(valid_clusters)} cụm")
        return valid_clusters
    
    def choose_cluster_head(self, cluster_nodes, cluster_ids, node_data=None):
        """
        Chọn cluster head
        - Ưu tiên: Node có năng lượng cao nhất
        - Dự phòng: Node gần tâm cụm nhất
        """
        if node_data:
            # Chọn theo năng lượng
            max_energy = -1
            ch_id = cluster_ids[0]
            
            for nid in cluster_ids:
                if nid in node_data and 'residual_energy' in node_data[nid]:
                    energy = node_data[nid]['residual_energy']
                    if energy > max_energy:
                        max_energy = energy
                        ch_id = nid
            
            return ch_id
        else:
            # Chọn theo khoảng cách đến tâm
            center = np.mean(cluster_nodes, axis=0)
            distances = np.linalg.norm(cluster_nodes - center, axis=1)
            min_idx = np.argmin(distances)
            return cluster_ids[min_idx]
    
    def calculate_metrics(self, clusters_data):
        """
        Tính các metric đánh giá chất lượng phân cụm
        """
        metrics = {
            'num_clusters': len(clusters_data),
            'avg_cluster_size': 0,
            'min_cluster_size': float('inf'),
            'max_cluster_size': 0,
            'avg_intra_distance': 0,
            'max_intra_distance': 0,
            'balance_score': 0  # Độ cân bằng kích thước cụm (0-1, càng cao càng tốt)
        }
        
        sizes = []
        intra_dists = []
        
        for nodes, ids in clusters_data:
            size = len(nodes)
            sizes.append(size)
            
            metrics['min_cluster_size'] = min(metrics['min_cluster_size'], size)
            metrics['max_cluster_size'] = max(metrics['max_cluster_size'], size)
            
            if size > 1:
                distances = pdist(nodes)
                intra_dists.append(np.mean(distances))
                metrics['max_intra_distance'] = max(metrics['max_intra_distance'], np.max(distances))
        
        metrics['avg_cluster_size'] = np.mean(sizes)
        metrics['avg_intra_distance'] = np.mean(intra_dists) if intra_dists else 0
        
        # Tính balance score (dựa trên coefficient of variation)
        cv = np.std(sizes) / np.mean(sizes) if np.mean(sizes) > 0 else 0
        metrics['balance_score'] = 1 / (1 + cv)  # 1 = hoàn toàn cân bằng
        
        return metrics

In [ ]:
def compute_vs(p1, p2, v_f, v_AUV):
    x1,y1,z1 = p1; x2,y2,z2 = p2
    Lx,Ly,Lz = x2-x1,y2-y1,z2-z1
    L_mag = math.sqrt(Lx**2+Ly**2+Lz**2)
    if L_mag == 0: return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta)/v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    if abs(cos_beta) < 1e-9: return v_AUV
    return abs(math.cos(angle) * v_AUV / cos_beta)

def travel_time(path, coords, v_f, v_AUV):
    if len(path) <= 1: return 0.0
    t = 0.0
    for i in range(len(path)-1):
        p1,p2 = coords[path[i]], coords[path[i+1]]
        d = np.linalg.norm(np.array(p2)-np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        t += d / max(v_s, 1e-9)
    p1,p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2)-np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    t += d / max(v_s, 1e-9)
    return t

def path_distance(path, coords):
    if len(path) <= 1: return 0.0
    dist = 0.0
    for i in range(len(path)-1):
        p1,p2 = coords[path[i]], coords[path[i+1]]
        dist += np.linalg.norm(np.array(p2)-np.array(p1))
    p1,p2 = coords[path[-1]], coords[path[0]]
    dist += np.linalg.norm(np.array(p2)-np.array(p1))
    return dist

In [ ]:
def get_swap_sequence(A, B):
    seq = []
    temp = A.copy()
    for i in range(1, len(A)): # Bắt đầu từ 1 vì 0 là cố định
        if temp[i] != B[i]:
            # Tìm vị trí của phần tử B[i] trong temp
            try:
                j = temp.index(B[i])
                seq.append((i, j))
                temp[i], temp[j] = temp[j], temp[i]
            except ValueError:
                # Xảy ra khi A và B không có cùng các phần tử (không nên xảy ra)
                pass 
    return seq

def apply_velocity(position, velocity):
    pos = position.copy()
    for i, j in velocity:
        if i > 0 and j > 0 and i < len(pos) and j < len(pos): # Bảo vệ index
            pos[i], pos[j] = pos[j], pos[i]
    return pos

def pso_tsp_3d_time(coords, v_f=1.0, v_AUV=3.0,
                    n_particles=40, max_iter=200,
                    w=0.7, c1=0.5, c2=0.5, init_gbest=None, verbose=True):
    
    n_cities = len(coords)
    if n_cities < 2: return [0], 0.0
    if n_cities == 2: return [0, 1], travel_time([0, 1], coords, v_f, v_AUV)
    
    cities = list(range(1, n_cities))

    # Khởi tạo quần thể
    swarm = [[0] + random.sample(cities, len(cities)) for _ in range(n_particles)]
    velocities = [[] for _ in range(n_particles)]

    # Đánh giá ban đầu
    costs = [travel_time(p, coords, v_f, v_AUV) for p in swarm]
    pbest = [p.copy() for p in swarm]
    pbest_cost = costs.copy()

    # Xác định gbest
    if init_gbest is not None and len(init_gbest) == n_cities:
        gbest = init_gbest.copy()
        gbest_cost = travel_time(gbest, coords, v_f, v_AUV)
    else:
        best_idx = int(np.argmin(pbest_cost))
        gbest = pbest[best_idx].copy()
        gbest_cost = pbest_cost[best_idx]

    # --- Vòng lặp chính ---
    for t in range(max_iter):
        inertia = 0.7 - 0.5 * (t / max_iter) # Giảm dần inertia

        for i in range(n_particles):
            xi, vi = swarm[i], velocities[i]

            # Giữ lại 1 phần vận tốc cũ
            v_new = vi[:int(inertia * len(vi))]

            # Ảnh hưởng cá nhân (pbest)
            if random.random() < c1:
                seq_pb = get_swap_sequence(xi, pbest[i])
                if seq_pb:
                    v_new += random.sample(seq_pb, min(2, len(seq_pb))) # Thêm 2 swap ngẫu nhiên

            # Ảnh hưởng toàn cục (gbest)
            if random.random() < c2:
                seq_gb = get_swap_sequence(xi, gbest)
                if seq_gb:
                    v_new += random.sample(seq_gb, min(2, len(seq_gb))) # Thêm 2 swap ngẫu nhiên

            # Cập nhật vị trí và vận tốc
            new_x = apply_velocity(xi, v_new)
            new_cost = travel_time(new_x, coords, v_f, v_AUV)
            
            swarm[i], velocities[i] = new_x, v_new

            # Cập nhật pbest và gbest
            if new_cost < pbest_cost[i]:
                pbest[i], pbest_cost[i] = new_x, new_cost
                if new_cost < gbest_cost:
                    gbest, gbest_cost = new_x, new_cost

        if verbose and t % 50 == 0:
            print(f"    [PSO Iter {t:3d}]: Best time = {gbest_cost:.4f}")

    if verbose:
         print(f"    [PSO Iter {max_iter:3d}]: Final Best time = {gbest_cost:.4f}")
    return gbest, gbest_cost

def multi_pso_tsp(coords, v_f=1.2, v_AUV=3.0, n_outer=5, verbose=True, **kwargs):
    """Chạy PSO nhiều lần để cải thiện hội tụ (Từ File 4)"""
    prev_gbest, prev_cost = None, float('inf')

    for outer in range(1, n_outer + 1):
        if verbose:
            print(f"   [PSO Outer loop {outer}/{n_outer}]")
            
        gbest, cost = pso_tsp_3d_time(coords, v_f=v_f, v_AUV=v_AUV,
                                      init_gbest=prev_gbest, verbose=verbose, **kwargs)
        
        if cost < prev_cost:
            prev_gbest, prev_cost = gbest, cost
        
        # Có thể thêm điều kiện dừng sớm nếu không cải thiện
        # if abs(cost - prev_cost) < 1e-6:
        #     print("   → Converged! Stop early.")
        #     break

    return prev_gbest, prev_cost

In [ ]:
def compute_energy(best_time, n_members):
    """
    Tính năng lượng tiêu thụ cho Member Node và Cluster Head.
    
    Parameters:
    - best_time: Thời gian hoàn thành chu kỳ AUV
    - n_members: Số lượng node thành viên thực tế trong cluster (không tính cluster head)
    """
    G, L = 100, 1024
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6

    # Năng lượng cho Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    # Năng lượng cho Cluster Head (nhận từ n_members node, truyền cho AUV)
    E_rx_TN = G * P_r * L * n_members / DR
    E_tx_TN = G * P_t * L * n_members / DR_i
    E_idle_TN = (best_time - (G*L*n_members/DR) - (G*L*n_members/DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_total": E_total_MN},
        "Target": {"E_total": E_total_TN}
    }

def update_energy(all_nodes, clusters, best_time):
    """
    Cập nhật năng lượng cho tất cả các node dựa trên số member thực tế của từng cluster.
    
    Parameters:
    - all_nodes: Dictionary chứa thông tin tất cả các node
    - clusters: Dictionary chứa thông tin các cluster
    - best_time: Thời gian hoàn thành chu kỳ AUV
    """
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        
        # Tính số member nodes (không tính cluster head)
        n_members = len([n for n in nodes if n != ch])
        
        # Tính năng lượng cho cluster này với số member thực tế
        energy_report = compute_energy(best_time, n_members)
        
        for nid in nodes:
            if nid not in all_nodes: continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)

def remove_dead_nodes(all_nodes, clusters):
    """
    Loại bỏ các node đã hết năng lượng và cập nhật lại clusters.
    
    Returns:
    - new_clusters: Dictionary các cluster còn node sống
    - dead: List các node_id đã chết
    """
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]

    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c

    return new_clusters, dead

In [ ]:
def recluster(all_nodes, node_positions, clustering_instance, r_sen=60, max_size=20, min_size=5):
    """
    Phân cụm lại toàn bộ các node còn sống sử dụng thuật toán từ cluster.py.
    
    Parameters:
    - all_nodes: Dictionary các node còn sống
    - node_positions: Dictionary vị trí của các node
    - clustering_instance: Instance của class Clustering
    - r_sen: Ngưỡng khoảng cách tối đa trong cụm
    - max_size: Số lượng node tối đa trong 1 cụm
    - min_size: Số lượng node tối thiểu trong 1 cụm
    
    Returns:
    - clusters: Dictionary các cụm mới
    """
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    clustering_instance.r_sen = r_sen
    clustering_instance.max_cluster_size = max_size
    clustering_instance.min_cluster_size = min_size
    clusters_data = clustering_instance.cluster_with_constraints(coords, ids)
    clusters = {}
    for i, (cluster_nodes, cluster_ids) in enumerate(clusters_data):
        center = np.mean(cluster_nodes, axis=0).tolist()
        ch = clustering_instance.choose_cluster_head(cluster_nodes, cluster_ids, all_nodes)
        clusters[i] = {'nodes': cluster_ids, 'center': center, 'cluster_head': ch}
    return clusters

In [ ]:
def reselect_cluster_heads(clusters, all_nodes):
    """
    Chỉ chọn lại cluster head cho các cụm hiện tại dựa trên năng lượng.
    Không phân cụm lại.
    """
    for cid, cinfo in clusters.items():
        cluster_ids = cinfo['nodes']
        # Tìm node có năng lượng cao nhất
        max_energy = -1
        new_ch = cluster_ids[0]
        for nid in cluster_ids:
            if nid in all_nodes and 'residual_energy' in all_nodes[nid]:
                energy = all_nodes[nid]['residual_energy']
                if energy > max_energy:
                    max_energy = energy
                    new_ch = nid
        clusters[cid]['cluster_head'] = new_ch
    return clusters

print("Helper function loaded")

In [ ]:
def main():
    """
    Mô phỏng PSO với enriched metadata Option B:
    - Reselect CH mỗi cycle; Recluster + PSO khi node chết.
    - Ghi path stats (seed vs best), clustering metrics, energy summaries, death analytics.
    """
    input_folder = "/kaggle/input/nodes-data"
    output_folder = "/kaggle/working/output_data_pso"
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(input_folder):
        print(f"❌ Lỗi: Thư mục {input_folder} không tồn tại!")
        return

    files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    if len(files) == 0:
        print(f"❌ Không tìm thấy file dữ liệu nào trong {input_folder}")
        return

    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0
    R_SEN = 60
    MAX_SIZE = 20
    MIN_SIZE = 5

    results_summary = []
    clustering = Clustering(space_size=400, r_sen=R_SEN, max_cluster_size=MAX_SIZE, min_cluster_size=MIN_SIZE)

    def seed_nearest_neighbor(coords):
        n = len(coords)
        if n <= 1: return [0]
        unvisited = set(range(1,n))
        path = [0]; cur = 0
        while unvisited:
            nxt = min(unvisited, key=lambda x: np.linalg.norm(np.array(coords[x]) - np.array(coords[cur])))
            path.append(nxt); unvisited.remove(nxt); cur = nxt
        return path

    for filename in files:
        input_path = os.path.join(input_folder, filename)
        print(f"\n{'='*60}\n=== Đang xử lý file: {filename} (PSO) ===\n{'='*60}")
        try:
            with open(input_path,'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc file {filename}: {e}")
            continue

        node_positions = {}
        all_nodes = {}
        if isinstance(data, list):
            for node in data:
                nid = node['id']
                all_nodes[nid] = {'initial_energy': node.get('initial_energy', INITIAL_ENERGY), 'residual_energy': node.get('residual_energy', INITIAL_ENERGY)}
                node_positions[nid] = (node['x'], node['y'], node['z'])
        else:
            print(f"❌ Cấu trúc file {filename} không được hỗ trợ")
            continue

        total_nodes = len(all_nodes)
        initial_total_energy = sum(nd['initial_energy'] for nd in all_nodes.values())
        print(f"Tổng số node ban đầu: {total_nodes}")

        print(f"\n{'='*60}\nPHÂN CỤM LẦN ĐẦU TIÊN\n{'='*60}")
        clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)

        initial_clusters_data_for_metrics = []
        for cid, cinfo in clusters.items():
            cluster_nodes = np.array([node_positions[nid] for nid in cinfo['nodes']])
            initial_clusters_data_for_metrics.append((cluster_nodes, cinfo['nodes']))
        initial_metrics = clustering.calculate_metrics(initial_clusters_data_for_metrics) if clusters else {'num_clusters':0,'avg_cluster_size':0,'min_cluster_size':0,'max_cluster_size':0,'avg_intra_distance':0,'max_intra_distance':0,'balance_score':0}

        sorted_keys = sorted(clusters.keys())
        centers = [(200,200,400)] + [tuple(clusters[k]['center']) for k in sorted_keys]
        center_coords = np.array(centers)

        # Seed path (nearest neighbor) for baseline
        seed_path = seed_nearest_neighbor(center_coords)
        seed_time = travel_time(seed_path, center_coords, v_f, v_AUV)
        seed_distance = path_distance(seed_path, center_coords)

        print(f"🌀 Chạy PSO tối ưu tour ban đầu...")
        best_path, best_time = multi_pso_tsp(center_coords, v_f, v_AUV, n_outer=3, n_particles=30, max_iter=80, verbose=False)
        best_distance = path_distance(best_path, center_coords)
        improvement_ratio = (seed_time - best_time)/seed_time if seed_time>0 else 0.0
        print(f"✅ Seed time={seed_time:.2f}s best time={best_time:.2f}s improve={improvement_ratio*100:.1f}% dist={best_distance:.2f}m clusters={len(clusters)}")

        cycle = 0
        alive_log = []
        energy_log = []
        cluster_count_log = []
        reclustering_cycles = []
        reclustering_details = []
        death_events = []
        reoptimization_count = 0
        last_seed_time = seed_time
        last_best_time = best_time
        last_best_distance = best_distance
        stop_reason = ""

        print(f"\n{'='*60}\n🚀 BẮT ĐẦU MÔ PHỎNG (PSO)\n{'='*60}")

        while True:
            cycle += 1
            alive_log.append(len(all_nodes))
            total_energy = sum(all_nodes[n]['residual_energy'] for n in all_nodes)
            energy_log.append(total_energy)
            cluster_count_log.append(len(clusters))

            alive_ratio = len(all_nodes)/total_nodes if total_nodes>0 else 0
            if alive_ratio < 0.9:
                stop_reason = f"Alive ratio below 90% ({alive_ratio*100:.2f}%)"
                print(f"\n🛑 Dừng ở cycle {cycle}: {stop_reason}")
                break

            print(f"\n--- Cycle {cycle} --- | Alive: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes}) | Energy: {total_energy:.2f}J | Clusters: {len(clusters)}")
            clusters = reselect_cluster_heads(clusters, all_nodes)

            update_energy(all_nodes, clusters, last_best_time)

            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            if dead_nodes:
                print(f"   ⚡ {len(dead_nodes)} node(s) chết: {dead_nodes}")
                death_events.append({'cycle': cycle,'dead_count': len(dead_nodes),'dead_ids': dead_nodes})
                if len(all_nodes) == 0:
                    stop_reason = "All nodes dead"
                    print("   ⚠️ Không còn node sống")
                    break
                print("   🔧 Phân cụm lại do node chết...")
                clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)
                if len(clusters) == 0:
                    stop_reason = "No clusters after reclustering"
                    print("   ⚠️ Không còn cụm hợp lệ")
                    break
                reclustering_cycles.append(cycle)
                reclustering_details.append({'cycle': cycle,'cluster_count': len(clusters)})
                sorted_keys = sorted(clusters.keys())
                centers = [(200,200,400)] + [tuple(clusters[k]['center']) for k in sorted_keys]
                center_coords = np.array(centers)
                seed_path_new = seed_nearest_neighbor(center_coords)
                seed_time_new = travel_time(seed_path_new, center_coords, v_f, v_AUV)
                best_path_new, best_time_new = multi_pso_tsp(center_coords, v_f, v_AUV, n_outer=3, n_particles=30, max_iter=80, verbose=False)
                best_distance_new = path_distance(best_path_new, center_coords)
                reoptimization_count += 1
                last_seed_time = seed_time_new
                last_best_time = best_time_new
                last_best_distance = best_distance_new
                print(f"   ✅ New seed={seed_time_new:.2f}s best={best_time_new:.2f}s improve={(seed_time_new-best_time_new)/seed_time_new*100 if seed_time_new>0 else 0:.1f}% dist={best_distance_new:.2f}m clusters={len(clusters)}")

        if not stop_reason:
            stop_reason = "Loop ended"

        total_dead_nodes = total_nodes - len(all_nodes)

        final_clusters_data_for_metrics = []
        for cid, cinfo in clusters.items():
            cluster_nodes = np.array([node_positions[nid] for nid in cinfo['nodes']])
            final_clusters_data_for_metrics.append((cluster_nodes, cinfo['nodes']))
        final_metrics = clustering.calculate_metrics(final_clusters_data_for_metrics) if clusters else {'num_clusters':0,'avg_cluster_size':0,'min_cluster_size':0,'max_cluster_size':0,'avg_intra_distance':0,'max_intra_distance':0,'balance_score':0}

        final_total_energy = sum(nd['residual_energy'] for nd in all_nodes.values())
        avg_energy_alive_final = final_total_energy/len(all_nodes) if all_nodes else 0.0
        per_cluster_energy_final = []
        for cid, cinfo in clusters.items():
            cluster_energy = sum(all_nodes[nid]['residual_energy'] for nid in cinfo['nodes'] if nid in all_nodes)
            per_cluster_energy_final.append({'cluster_id': cid,'nodes': cinfo['nodes'],'cluster_head': cinfo['cluster_head'],'total_energy': cluster_energy})

        dead_node_ids = sorted({nid for ev in death_events for nid in ev['dead_ids']})
        death_cycle_counts = {ev['cycle']: ev['dead_count'] for ev in death_events}
        first_death_cycle = death_events[0]['cycle'] if death_events else None
        last_death_cycle = death_events[-1]['cycle'] if death_events else None
        mean_deaths_per_event = (total_dead_nodes/len(death_events)) if death_events else 0.0
        cycles_until_first_death = first_death_cycle if first_death_cycle is not None else None

        meta = {
            'input_file': filename,
            'method': 'PSO Selective Reclustering',
            'strategy': 'Reselect CH each cycle; Recluster+PSO on node death',
            'parameters': {
                'r_sen': R_SEN,
                'max_cluster_size': MAX_SIZE,
                'min_cluster_size': MIN_SIZE,
                'v_flow': v_f,
                'v_AUV': v_AUV,
                'pso_particles': 30,
                'pso_outer_loops': 3,
                'pso_max_iter': 80
            },
            'lifecycle': {
                'cycles_completed': cycle-1,
                'stop_reason': stop_reason,
                'reoptimization_count': reoptimization_count,
                'reclustering_cycles': reclustering_cycles,
                'reclustering_details': reclustering_details
            },
            'nodes_summary': {
                'initial_total_nodes': total_nodes,
                'final_alive_nodes': len(all_nodes),
                'final_alive_ratio': len(all_nodes)/total_nodes if total_nodes>0 else 0,
                'total_dead_nodes': total_dead_nodes,
                'dead_node_ids': dead_node_ids
            },
            'path_stats': {
                'initial_seed_time': seed_time,
                'initial_best_time': best_time,
                'initial_improvement_ratio': improvement_ratio,
                'final_seed_time': last_seed_time,
                'final_best_time': last_best_time,
                'final_improvement_ratio': (last_seed_time - last_best_time)/last_seed_time if last_seed_time>0 else 0.0,
                'initial_seed_distance': seed_distance,
                'initial_best_distance': best_distance,
                'final_best_distance': last_best_distance
            },
            'clustering_initial': initial_metrics,
            'clustering_final': final_metrics,
            'energy': {
                'initial_total_energy': initial_total_energy,
                'final_total_energy': final_total_energy,
                'avg_energy_alive_final': avg_energy_alive_final,
                'per_cluster_energy_final': per_cluster_energy_final
            },
            'death_events': death_events,
            'death_analytics': {
                'death_cycle_counts': death_cycle_counts,
                'first_death_cycle': first_death_cycle,
                'last_death_cycle': last_death_cycle,
                'mean_deaths_per_event': mean_deaths_per_event,
                'cycles_until_first_death': cycles_until_first_death
            }
        }

        output_json = os.path.join(output_folder, f"result_{filename}")
        with open(output_json,'w') as f: json.dump(meta, f, indent=4)

        results_summary.append((filename, cycle-1, len(reclustering_cycles), total_dead_nodes))
        print(f"\n✅ {filename}: cycles={cycle-1}, recluster={len(reclustering_cycles)}, dead={total_dead_nodes}")

        plt.figure(figsize=(12,8))
        plt.subplot(2,2,1)
        plt.plot(range(len(alive_log)), alive_log, marker='o', linewidth=2, color='steelblue')
        for rc in reclustering_cycles: plt.axvline(x=rc, color='red', linestyle='--', alpha=0.5)
        plt.title(f"Alive nodes - {filename}"); plt.xlabel("Cycle"); plt.ylabel("Alive"); plt.grid(True, alpha=0.3)
        plt.axhline(y=total_nodes*0.9, color='red', linestyle='--', linewidth=2)

        plt.subplot(2,2,2)
        plt.plot(range(len(energy_log)), energy_log, marker='s', linewidth=2, color='orange')
        for rc in reclustering_cycles: plt.axvline(x=rc, color='red', linestyle='--', alpha=0.5)
        plt.title(f"Total energy - {filename}"); plt.xlabel("Cycle"); plt.ylabel("Energy (J)"); plt.grid(True, alpha=0.3)

        plt.subplot(2,2,3)
        plt.plot(range(1, len(cluster_count_log)+1), cluster_count_log, marker='^', linewidth=2, color='green')
        for rc in reclustering_cycles: plt.axvline(x=rc, color='red', linestyle='--', alpha=0.5)
        plt.title(f"Clusters per cycle - {filename}"); plt.xlabel("Cycle"); plt.ylabel("Clusters"); plt.grid(True, alpha=0.3)

        plt.subplot(2,2,4)
        plt.axis('off')
        alive_ratio_final = len(all_nodes)/total_nodes if total_nodes>0 else 0
        death_lines = []
        for ev in death_events[:6]: death_lines.append(f"  • C{ev['cycle']}: -{ev['dead_count']}")
        if len(death_events) > 6: death_lines.append("  • ...")
        death_block = "\n".join(death_lines) if death_lines else "  • (None)"
        info_text = f"""SUMMARY\nCycles: {cycle-1}\nRecluster events: {len(reclustering_cycles)}\nTotal dead: {total_dead_nodes}\nAlive: {len(all_nodes)}/{total_nodes} ({alive_ratio_final*100:.1f}%)\nStop: {stop_reason}\nDeaths:\n{death_block}\nInitial improve: {improvement_ratio*100:.1f}%\nFinal improve: {(last_seed_time-last_best_time)/last_seed_time*100 if last_seed_time>0 else 0:.1f}%"""
        plt.text(0.02, 0.05, info_text, fontsize=9, family='monospace', va='bottom')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f"summary_{filename}.png"), dpi=150)
        plt.close()

    if results_summary:
        print(f"\n{'='*60}\n📊 TỔNG KẾT\n{'='*60}")
        for fname, cycles, recount, dead in results_summary:
            print(f"  {fname}: cycles={cycles}, recluster={recount}, dead={dead}")
        print(f"\n✅ Hoàn thành! Kết quả tại: {output_folder}")
    else:
        print("\n❌ Không có kết quả nào được tạo ra")

print("✓ Main function (PSO enriched) loaded")

In [ ]:
if __name__ == '__main__':
    main()